In [ ]:
#### Pipeline

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
from itertools import combinations
from xgboost import XGBClassifier
import re
from pycaret.classification import setup, compare_models, create_model, tune_model, plot_model, evaluate_model, finalize_model, predict_model, save_model, load_model, get_config
import warnings
warnings.filterwarnings("ignore")

In [6]:
path = "C:/Users/danie/Documents/Parcial_Python/"

In [ ]:
%%time
ba_entr = pd.read_csv(path  + 'train.csv')
ba_pru = pd.read_csv(path + "test.csv")

In [ ]:
ba_entr

In [ ]:
category_mapping = {'Graduate': 0, 'Dropout': 1, 'Enrolled': 2}
ba_entr['Target'] = ba_entr['Target'].map(category_mapping)
ba_entr['Target'] = ba_entr['Target'].astype('object')

In [ ]:
ct = ['Marital status', 'Application mode', 'Application order',
       'Course', 'Daytime/evening attendance', 'Previous qualification',
       'Nacionality', "Mother's qualification", "Father's qualification",
       "Mother's occupation", "Father's occupation", 'Displaced', 
       'Educational special needs', 'Debtor','Tuition fees up to date', 
       'Gender', 'Scholarship holder', 'International']

for k in ct:
  ba_entr[k] = ba_entr[k].astype("O")
  ba_pru[k] = ba_pru[k].astype("O")

In [ ]:
formato = pd.DataFrame({'Variable': list(ba_entr.columns),'Formato': ba_entr.dtypes })
ft = pd.merge(ft,formato,on=["Variable"],how="left")
ft.head()

In [ ]:
cuantitativas = list(formato.loc[formato["Formato"]!="object","Variable"])
cuantitativas = [x for x in cuantitativas if x not in ["id","Target"]]

In [ ]:
for k in cuantitativas:
    plt.figure(figsize=(8,6))
    sns.boxplot(x="Target",y=k,data=ba_entr)
    plt.title("Boxplot de "+k+" vs Target")

Los diferentes boxplot muestran una comparación de las calificaciones previas entre tres grupos definidos por la variable Target (estado del estudiante: graduado, matriculado y retirado). Grade vs Target: A pesar de que existen algunos valores atípicos en cada grupo, en general, las calificaciones previas se distribuyen de manera similar entre los tres grupos. La mediana y el rango intercuartílico son bastante parecidos, lo que sugiere que no hay diferencias significativas en el rendimiento académico previo entre estos grupos.

Admission grade vs Target: Se puede identificar que las calificaciones de admisión se distribuyen de manera similar entre los tres grupos del "Target", con una mediana y un rango intercuartílico similares. Sin embargo, existen algunos valores atípicos con puntuaciones muy altas o muy bajas en cada grupo. Hay casos individuales que se desvían significativamente de la media.

Enrollement vs Target: Por otra parte en este boxplot se puede evidenciar que la edad de inscripción es más baja en el grupo 0 y ligeramente más alta en los grupos 1 y 2, siendo 1 el grupo con mayor rango intercuartílico. Sin embargo, existe una gran variabilidad en la edad de inscripción dentro de cada grupo, con algunos individuos inscribiéndose a edades significativamente mayores. Los valores atípicos (puntos individuales por encima y por debajo de las cajas) sugieren la presencia de casos excepcionales en cada grupo. Es decir, aunque existe una tendencia general de edades más altas en los grupos 1 y 2 comparado con el grupo 0, la variabilidad es considerable.

1st sem credited vs Target: No hay una gran diferencia en el número de unidades curriculares acreditadas en el primer semestre entre los tres grupos del "Target".

1st sem enrolled vs Target: A diferencia del anterior boxplot el número de unidades curriculares en las que se inscribieron los estudiantes es similar en los tres grupos del "Target", con una mediana y un rango intercuartílico similares. A diferencia del anterior los valores atípicos toman valores muy altos o muy bajos en cada grupo.

1st evaluation vs Target: El boxplot indica que el número de unidades curriculares evaluadas no parece ser un factor determinante para distinguir entre los tres grupos del "Target", al menos basándonos en esta visualización.

1st sem approved vs Target: Este boxplot nos deja evidenciar que se encuentran valores atípicos tanto altos como bajos a diferencia de la categoria 1 y 2 los cuales presentan una ligera similitud en el rango intercuartílico.

1st sem grade vs Target: En os grupos 0 y 2 se evidencia una distribución similar, con una mediana de alrededor de 5 unidades curriculares aprobadas y varios valores atípicos, lo que indica que algunos estudiantes en estos grupos aprobaron significativamente más unidades que el promedio. En cambio, el grupo 1 tiene una mediana más baja, cerca de 3 unidades curriculares, y menos valores atípicos, con una dispersión más concentrada alrededor de la mediana. La dispersión es mayor en los grupos 0 y 2, reflejando una mayor variabilidad en las unidades aprobadas en comparación con el grupo 1.

1 st sem without evaluation vs Target: Se observa una mayor similitud entre los tres grupos, con medianas cercanas y una dispersión más homogénea, representada por un rango intercuartílico estrecho y valores atípicos menos destacados. Esto sugiere una percepción diferente de la variabilidad y la similitud entre los grupos dependiendo de la interpretación.

2nd sem credited vs Target: En este grafico podemos observar que en las tres categorias se encuentra una dispersión similar con valores atipicos hasta 17.5 unidades curriculares.

2nd sem enrolled vs Target: En los tres grupos se presentan una distribución bastante similar, con la mayoría de los estudiantes matriculados entre 5 y 10 unidades curriculares. Las medianas de los grupos son cercanas, lo que indica que, en promedio, los estudiantes se matricularon en una cantidad similar de unidades. El rango intercuartílico también es parecido entre los grupos, lo que sugiere que los datos están concentrados en un rango pequeño. Se observan algunos valores atípicos en cada grupo, representando a aquellos estudiantes que se matricularon en un número significativamente mayor de unidades en comparación con sus compañeros.

2nd sem evaluation vs Target:Se logra evidenciar que en la categoría 1 las unidades curriculares evaluadas estan entre 0 y 9 unidades curriculares a comparación de la categoría 0 y 2 las cuales tienen una dispersión similar de los datos, sin embargo en las tres categorias se encuentran valores atípicos que representan aquellos estudiantes evaluados.

2nd sem grade vs Target: Los tres grupos presentan distribuciones bastante similares en cuanto a las calificaciones de unidades curriculares, con la mayoría de los estudiantes obteniendo calificaciones en un rango de 10 a 15 unidades. Aunque hay algunos valores atípicos que representan estudiantes con calificaciones significativamente mayores, el rango intercuartílico es comparable en los tres grupos, indicando que los datos están concentrados en un rango reducido. Las medianas de los grupos son muy cercanas, lo que sugiere que, en promedio, los estudiantes de los tres grupos obtienen calificaciones en una cantidad similar de unidades.

2nd sem without evaluation vs Target: En el gráfico se observa que las tres categorías exhiben una dispersión similar, con algunos valores atípicos que se extienden hasta 12 unidades curriculares.

Unemployement vs Target: Se puede evidenciar que los tres grupos tienen distribuciones similares de la tasa de desempleo, con la mayoría de los datos ubicados entre un 8% y un 16%. Las medianas son casi idénticas, indicando tasas promedio similares entre los grupos, y el rango intercuartílico es comparable, lo que sugiere una dispersión similar en torno a la mediana. No hay valores atípicos significativos, lo que refuerza la idea de que no hay diferencias notables en la tasa de desempleo entre los grupos.

Inflation vs Target: El boxplot revela que los tres grupos tienen una distribución bastante similar en cuanto a la tasa de inflación, con la mayoría de los datos situados entre el 0% y el 3%. Las medianas de los grupos son muy cercanas, lo que indica tasas promedio similares, y el rango intercuartílico es comparable, sugiriendo una dispersión similar en torno a la mediana. No se observan valores atípicos significativos, lo que refuerza la idea de que no hay diferencias notables en la tasa de inflación entre los grupos.

GDP vs Target: El ultimo boxplot indica que los tres grupos presentan una distribución comparable en términos de crecimiento del PIB, con la mayoría de los valores ubicados entre -2% y 2%. Las medianas son casi idénticas, lo que sugiere tasas promedio de crecimiento similares, y el rango intercuartílico muestra una dispersión uniforme alrededor de la mediana. La ausencia de valores atípicos significativos refuerza la idea de que no hay diferencias importantes en el crecimiento del PIB entre los grupos. En conjunto, el crecimiento del PIB se comporta de manera homogénea en los tres grupos.

In [ ]:
def asociacion(data):
    categoricas=data.columns[data.dtypes=='O']
    V1=np.array(categoricas); V2=np.array(categoricas)
    grilla=np.meshgrid(V1,V2)
    grilla=pd.DataFrame({'Var1':grilla[0].ravel(),'Var2':grilla[1].ravel()})
    p_value=[stats.chi2_contingency(pd.DataFrame(pd.crosstab(data[grilla.iloc[x]['Var1']],data[grilla.iloc[x]['Var2']])))[1] for x in range(grilla.shape[0])]

    grilla['p_value']=p_value
    grilla2=grilla.pivot(index='Var1',columns='Var2',values='p_value')
    plt.figure(figsize=(10,8))
    gr=sns.heatmap(grilla2,linewidths=0.01,annot=True,fmt='.2f',cmap='summer')
    gr.set_title('Grilla de p valores en prueba chi cuadrado para verificar asociación entre variables')
    plt.xlabel(""); plt.ylabel(""); plt.yticks(rotation=0); plt.xticks(rotation=90)
    plt.show()
    return grilla,grilla2

In [ ]:
grilla_asociacion, grilla_asociacion2 = asociacion(ba_entr.drop(["id"],axis=1))

Un p-valor cercano a 0 indica una evidencia de que las variables no están asociadas, es decir, las categorías de una variable son independientes de las categorías de la otra. En contraste, un p-valor cercano a 1 sugiere que las variables son dependientes, mostrando una fuerte evidencia de asociación. La gráfica se presenta en forma de una matriz de colores, donde los tonos amarillos indican p-valores altos mayor evidencia de asociación y los tonos verdes p-valores altos menor evidencia de asociación.

Se puede evidenciar que la mayoria de las variables no tienen una asociación dado que encontramos en gran proporción ceros en la matriz, exceptuando una fuerte asociación entre variables como: Nacionalidad y ocupación del padre. Nacionalidad y ocupación de la madre. Nacionalidad y calificación previa.



In [49]:
class ML_FLOW_PARCIAL:
    def __init__(self, met, mod):
        self.met = met #selecciona si el modelo es con o sin ing. de variables
        self.mod = mod #selecciona alguno de los tres mejores modelos
        pass
        
    def load_data(self):
        ba_entr = pd.read_csv(path  + 'train.csv')
        ba_pru = pd.read_csv(path + "test.csv")
        return ba_entr, ba_pru
        
    def preprocessing(self):
        ##TRATAMIENTO DE DATOS
        category_mapping = {'Graduate': 0, 'Dropout': 1, 'Enrolled': 2}
        
        if self.met == True:
            self.ba_entr['Target'] = self.ba_entr['Target'].map(category_mapping).astype('object')
        else:
            self.ba_entr['Target'] = self.ba_entr['Target'].map(category_mapping)
            
        ct = ['Marital status', 'Application mode', 'Application order',
        'Course', 'Daytime/evening attendance', 'Previous qualification',
        'Nacionality', "Mother's qualification", "Father's qualification",
        "Mother's occupation", "Father's occupation", 'Displaced', 
        'Educational special needs', 'Debtor','Tuition fees up to date', 
        'Gender', 'Scholarship holder', 'International']
        
        for k in ct:
            self.ba_entr[k] = self.ba_entr[k].astype("O")
            self.ba_pru[k] = self.ba_pru[k].astype("O")
        
        return self.ba_entr, self.ba_pru
            
    def train_model(self):

        formato = pd.DataFrame({'Variable': list(self.ba_entr.columns), 'Formato': self.ba_entr.dtypes })
        
        cuantitativas = list(formato.loc[formato["Formato"]!="object","Variable"])
        cuantitativas = [x for x in cuantitativas if x not in ["id","Target"]]
        
        if self.met == True:
        
            ## Variables al cuadrado
            base_cuadrado = self.ba_entr.get(cuantitativas).copy()
            base_cuadrado["Target"] = self.ba_entr["Target"].copy()

            var_names2, pvalue1 = [], []

            for k in cuantitativas:
                base_cuadrado[k+"_2"] = base_cuadrado[k] ** 2

                # Prueba de Kruskal sin logaritmo
                mue1 = base_cuadrado.loc[base_cuadrado["Target"]==0,k+"_2"].to_numpy()
                mue2 = base_cuadrado.loc[base_cuadrado["Target"]==1,k+"_2"].to_numpy()
                mue3 = base_cuadrado.loc[base_cuadrado["Target"]==2,k+"_2"].to_numpy()

                p1 = stats.kruskal(mue1,mue2,mue3)[1]

                # Guardar p values y variables
                var_names2.append(k+"_2")
                pvalue1.append(np.round(p1,2))
            pcuadrado1 = pd.DataFrame({'Variable2':var_names2,'p value':pvalue1})
            pcuadrado1["criterio"] = pcuadrado1.apply(lambda row: 1 if row["p value"]<=0.10 else 0,axis = 1)

            ## Interacciones cuantitativas
            lista_inter = list(combinations(cuantitativas,2))
            base_interacciones = self.ba_entr.get(cuantitativas).copy()
            var_interaccion, pv1 = [], []
            base_interacciones["Target"] = self.ba_entr["Target"].copy()

            for k in lista_inter:
                base_interacciones[k[0]+"__"+k[1]] = base_interacciones[k[0]] * base_interacciones[k[1]]

                # Prueba de Kruskal
                mue1 = base_interacciones.loc[base_interacciones["Target"]==0,k[0]+"__"+k[1]].to_numpy()
                mue2 = base_interacciones.loc[base_interacciones["Target"]==1,k[0]+"__"+k[1]].to_numpy()
                mue3 = base_interacciones.loc[base_interacciones["Target"]==2,k[0]+"__"+k[1]].to_numpy()
                p1 = stats.kruskal(mue1,mue2,mue3)[1]

                var_interaccion.append(k[0]+"__"+k[1])
                pv1.append(np.round(p1,2))
            pxy = pd.DataFrame({'Variable':var_interaccion,'p value':pv1})
            pxy["criterio"] = pxy.apply(lambda row: 1 if row["p value"]<=0.10 else 0, axis = 1)
            
            ## Razones
            raz1 = [(x,y) for x in cuantitativas for y in cuantitativas]
            base_razones1 = self.ba_entr.get(cuantitativas).copy()
            base_razones1["Target"] = self.ba_entr["Target"].copy()

            var_nm, pval = [], []
            for j in raz1:
                if j[0]!=j[1]:
                    base_razones1[j[0]+"__coc__"+j[1]] = base_razones1[j[0]] / (base_razones1[j[1]]+0.01)

                    # Prueba de Kruskal
                    mue1 = base_razones1.loc[base_razones1["Target"]==0,j[0]+"__coc__"+j[1]].to_numpy()
                    mue2 = base_razones1.loc[base_razones1["Target"]==1,j[0]+"__coc__"+j[1]].to_numpy()
                    mue3 = base_razones1.loc[base_razones1["Target"]==2,j[0]+"__coc__"+j[1]].to_numpy()
                    p1 = stats.kruskal(mue1,mue2,mue3)[1]
        
                    # Guardar valores
                    var_nm.append(j[0]+"__coc__"+j[1])
                    pval.append(np.round(p1,2))
            prazones = pd.DataFrame({'Variable':var_nm,'p value':pval})
            prazones["criterio"] = prazones.apply(lambda row: 1 if row["p value"]<=0.10 else 0, axis = 1)
            
            ## Interacciones categóricas
            categoricas = list(formato.loc[formato["Formato"]=="O","Variable"])
            categoricas = [x for x in categoricas if x not in ["id","Target"]]

            def nombre_(x):
              return "C"+str(x)
            cb = list(combinations(categoricas,2))
            p_value, modalidades, nombre_var = [], [], []

            base2 = self.ba_entr.get(categoricas).copy()
            for k in base2.columns:
              base2[k] = base2[k].map(nombre_)

            base2["Target"] = self.ba_entr["Target"].copy()

            for k in range(len(cb)):
                # Variable con interacción
                base2[cb[k][0]] = base2[cb[k][0]]
                base2[cb[k][1]] = base2[cb[k][1]]

                base2[cb[k][0]+"__"+cb[k][1]] = base2[cb[k][0]] + "__" + base2[cb[k][1]]

                # Prueba chi cuadrado
                c1 = pd.DataFrame(pd.crosstab(base2["Target"],base2[cb[k][0]+"__"+cb[k][1]]))
                pv = stats.chi2_contingency(c1)[1]

                # Número de modalidades por categoría
                mod_ = len(base2[cb[k][0]+"__"+cb[k][1]].unique())

                # Guardar p value y modalidades
                nombre_var.append(cb[k][0]+"__"+cb[k][1])
                modalidades.append(mod_)
                p_value.append(pv)
            pc = pd.DataFrame({'Variable':nombre_var,'Num Modalidades':modalidades,'p value':p_value})
            pc.loc[(pc["p value"]<=0.20) & (pc["Num Modalidades"]<=8),].sort_values(["p value"],ascending=True)
            ## Dummies categóricas más significativas (p value <= 0.20 y bajo número de modalidades)
            def indicadora(x):
              if x==True:
                return 1
              else:
                return 0

            seleccion1 = list(pc.loc[(pc["p value"]<=0.20) & (pc["Num Modalidades"]<=8),"Variable"])
            sel1 = base2.get(seleccion1)


            ## convertir categorica a numerica: dummies
            contador = 0
            lb1 = pd.DataFrame()
            for k in sel1:
                if contador==0:
                    lb1 = pd.get_dummies(sel1[k],drop_first=True)
                    lb1.columns = [k + "_" + x for x in lb1.columns]
                else:
                    lb2 = pd.get_dummies(sel1[k],drop_first=True)
                    lb2.columns = [k + "_" + x for x in lb2.columns]
                    lb1 = pd.concat([lb1,lb2],axis=1)
                    lb1
                contador = contador + 1
            
            for k in lb1.columns:
              lb1[k] = lb1[k].map(indicadora)

            lb1["Target"] = self.ba_entr["Target"].copy()

            ## Interacción cuantitativa vs categórica
            cat_cuanti = [(x,y) for x in cuantitativas for y in categoricas]
            v1, v2, pvalores_min, pvalores_max  = [], [], [], []

            for j in cat_cuanti:
                k1 = j[0]
                k2 = j[1]

                g1 = pd.get_dummies(self.ba_entr[k2])
                lt1 = list(g1.columns)

                for k in lt1:
                    g1[k] = g1[k] * self.ba_entr[k1]

                g1["Target"] = self.ba_entr["Target"].copy()
                pvalues_c = []
                for y in lt1:
                    mue1 = g1.loc[g1["Target"]==0,y].to_numpy()
                    mue2 = g1.loc[g1["Target"]==1,y].to_numpy()
                    mue3 = g1.loc[g1["Target"]==2,y].to_numpy()
        
                    try:
                      pval = (stats.kruskal(mue1,mue2,mue3)[1]<=0.20)

                      if pval==True:
                          pval = 1
                      else:
                          pval = 0
                    except ValueError:
                      pval = 0
                    pvalues_c.append(pval)

                min_ = np.min(pvalues_c) # Se revisa si alguna de las categorías no es significativa
                max_ = np.max(pvalues_c) # Se revisa si alguna de las categorías es significativa
                v1.append(k1) # nombre de la variable 1
                v2.append(k2) # nombre de la variable 2
                pvalores_min.append(np.round(min_,2))
                pvalores_max.append(np.round(max_,2))
            pc2 = pd.DataFrame({'Cuantitativa':v1,'Categórica':v2,'p value':pvalores_min, 'p value max':pvalores_max})
            pc2.loc[(pc2["p value"]==1) & (pc2["p value max"]==1),]

            ## Base de Feature Enginnering
            v1 = list(pc2.loc[(pc2["p value"]==1) & (pc2["p value max"]==1),"Cuantitativa"])
            v2 = list(pc2.loc[(pc2["p value"]==1) & (pc2["p value max"]==1),"Categórica"])

            for j in range(len(v1)):

                if j==0:
                    g1 = pd.get_dummies(self.ba_entr[v2[j]],drop_first=True)
                    lt1 = list(g1.columns)
                    for k in lt1:
                        g1[k] = g1[k] * self.ba_entr[v1[j]]
                    g1.columns = [v1[j] + "_" + v2[j] + "_" + str(x) for x in lt1]
                else:
                    g2 = pd.get_dummies(self.ba_entr[v2[j]],drop_first=True)
                    lt1 = list(g2.columns)
                    for k in lt1:
                        g2[k] = g2[k] * self.ba_entr[v1[j]]
                    g2.columns = [v1[j] + "_" + v2[j] + "_" + str(x) for x in lt1]
                    g1 = pd.concat([g1,g2],axis=1)

            g1["Target"] = self.ba_entr["Target"].copy()
            #### SELECCION DE VARIABLES CON XGBOOST
            var_cuad = list(pcuadrado1["Variable2"])
            base_modelo1 = base_cuadrado.get(var_cuad+["Target"])
            base_modelo1["Target"] = base_modelo1["Target"].map(int) ## convertir en enteros las categorias
            cov = list(base_modelo1.columns)
            cov = [x for x in cov if x not in ["Target"]]

            X1 = base_modelo1.get(cov)
            y1 = base_modelo1.get(["Target"])

            modelo1 = XGBClassifier()
            modelo1 = modelo1.fit(X1,y1)

            importancias = modelo1.feature_importances_
            imp1 = pd.DataFrame({'Variable':X1.columns,'Importancia':importancias})
            imp1["Importancia"] = imp1["Importancia"] * 100 / np.sum(imp1["Importancia"])
            imp1 = imp1.sort_values(["Importancia"],ascending=False)
            imp1.index = range(imp1.shape[0])

            var_int = list(pxy["Variable"])
            base_modelo2 = base_interacciones.get(var_int+["Target"])
            base_modelo2["Target"] = base_modelo2["Target"].map(int)
            cov = list(base_modelo2.columns)
            cov = [x for x in cov if x not in ["Target"]]

            X2 = base_modelo2.get(cov)
            y2 = base_modelo2.get(["Target"])

            modelo2 = XGBClassifier()
            modelo2 = modelo2.fit(X2,y2)

            importancias = modelo2.feature_importances_
            imp2 = pd.DataFrame({'Variable':X2.columns,'Importancia':importancias})
            imp2["Importancia"] = imp2["Importancia"] * 100 / np.sum(imp2["Importancia"])
            imp2 = imp2.sort_values(["Importancia"],ascending=False)
            imp2.index = range(imp2.shape[0])

            var_raz = list(prazones["Variable"])
            base_modelo3 = base_razones1.get(var_raz+["Target"])
            base_modelo3["Target"] = base_modelo3["Target"].map(int)
            cov = list(base_modelo3.columns)
            cov = [x for x in cov if x not in ["Target"]]

            X3 = base_modelo3.get(cov)
            y3 = base_modelo3.get(["Target"])

            modelo3 = XGBClassifier()
            modelo3 = modelo3.fit(X3,y3)

            importancias = modelo3.feature_importances_
            imp3 = pd.DataFrame({'Variable':X3.columns,'Importancia':importancias})
            imp3["Importancia"] = imp3["Importancia"] * 100 / np.sum(imp3["Importancia"])
            imp3 = imp3.sort_values(["Importancia"],ascending=False)
            imp3.index = range(imp3.shape[0])

            lb1["Target"] = lb1["Target"].map(int)
            cov = list(lb1.columns)
            cov = [x for x in cov if x not in ["Target"]]

            X4 = lb1.get(cov)
            y4 = lb1.get(["Target"])

            modelo4 = XGBClassifier()
            modelo4 = modelo4.fit(X4,y4)

            importancias = modelo4.feature_importances_
            imp4 = pd.DataFrame({'Variable':X4.columns,'Importancia':importancias})
            imp4["Importancia"] = imp4["Importancia"] * 100 / np.sum(imp4["Importancia"])
            imp4 = imp4.sort_values(["Importancia"],ascending=False)
            imp4.index = range(imp4.shape[0])

            g1["Target"] = g1["Target"].map(int)
            cov = list(g1.columns)
            cov = [x for x in cov if x not in ["Target"]]

            X5 = g1.get(cov)
            y5 = g1.get(["Target"])

            modelo5 = XGBClassifier()
            modelo5 = modelo5.fit(X5,y5)

            importancias = modelo5.feature_importances_
            imp5 = pd.DataFrame({'Variable':X5.columns,'Importancia':importancias})
            imp5["Importancia"] = imp5["Importancia"] * 100 / np.sum(imp5["Importancia"])
            imp5 = imp5.sort_values(["Importancia"],ascending=False)
            imp5.index = range(imp5.shape[0])

            #### VARIABLES MAS IMPORTANTES POR XGBOOST EN CADA CASO ####
            c2 = list(imp1.iloc[0:3,0]) # Variables al cuadrado
            cxy = list(imp2.iloc[0:3,0]) # Interacciones cuantitativas
            razxy = list(imp3.iloc[0:3,0]) # Razones
            catxy = list(imp4.iloc[0:3,0]) # Interacciones categóricas
            cuactxy = list(imp5.iloc[0:3,0]) # Interacción cuantitativa y categórica

            # Variables cuantitativas (Activar D1)
            D1 = self.ba_entr.get(cuantitativas).copy()

            # Variables categóricas
            D2 = self.ba_entr.get(categoricas).copy()
            for k in categoricas:
              D2[k] = D2[k].map(nombre_)
            D4 = D2.copy()

            # Variables al cuadrado (Activar D1)
            cuadrado = [re.findall(r'(.+)_\d+', item) for item in c2]
            cuadrado = [x[0] for x in cuadrado]

            for k in cuadrado:
              D1[k+"_2"] = D1[k] ** 2
            
            # Interacciones cuantitativas (Activar D1)
            result = [re.findall(r'([A-Za-z\s\(\)0-9]+)', item) for item in cxy]

            for k in result:
              D1[k[0]+"__"+k[1]] = D1[k[0]] * D1[k[1]]

            # Razones
            result2 = [re.findall(r'(.+)__coc__(.+)', item) for item in razxy]
            for k in result2:
              k2 = k[0]
              D1[k2[0]+"__coc__"+k2[1]] = D1[k2[0]] / (D1[k2[1]]+0.01)

            # Interacciones categóricas
            result3 = [re.search(r'([^_]+__[^_]+)', item).group(1).split('__') for item in catxy]
            for k in result3:
              D4[k[0]+"__"+k[1]] = D4[k[0]] + "_" + D4[k[1]]

            # Interacción cuantitativa vs categórica
            D5 = self.ba_entr.copy()
            result4 = [re.search(r'(.+?)_(.+?)_1', item).groups() for item in cuactxy]
            contador = 0
            for k in result4:
              col1, col2 = k[1], k[0] # categórica, cuantitativa
              if contador == 0:
                D51 = pd.get_dummies(D5[col1],drop_first=True)
                for j in D51.columns:
                  D51[j] = D51[j] * D5[col2]
                D51.columns = [col2+"_"+col1+"_"+ str(x) for x in D51.columns]
              else:
                D52 = pd.get_dummies(D5[col1],drop_first=True)
                for j in D52.columns:
                  D52[j] = D52[j] * D5[col2]
                D52.columns = [col2+"_"+col1+"_"+ str(x) for x in D52.columns]
                D51 = pd.concat([D51,D52],axis=1)
              contador = contador + 1
            #### BASE MODELO ####
            B1 = pd.concat([D1,D4],axis=1)
            base_modelo = pd.concat([B1,D51],axis=1)
            base_modelo["Target"] = self.ba_entr["Target"].copy()
            base_modelo["Target"] = base_modelo["Target"].map(int)

            #### AUTOML ####
            formatos = pd.DataFrame(base_modelo.dtypes).reset_index()
            formatos.columns = ["Variable","Formato"]
            cuantitativas_bm = list(formatos.loc[formatos["Formato"]!="object",]["Variable"])
            categoricas_bm = list(formatos.loc[formatos["Formato"]=="object",]["Variable"])
            cuantitativas_bm = [x for x in cuantitativas_bm if x not in ["Target"]]
            categoricas_bm = [x for x in categoricas_bm if x not in ["Target"]]

            # Configuración del experimento
            exp_clf101 = setup(data=base_modelo,
            target='Target',
            session_id=123,
            train_size=0.7,
            numeric_features = cuantitativas_bm,
            categorical_features = categoricas_bm,
            fix_imbalance=False)
        else:
            formatos = pd.DataFrame(self.ba_entr.dtypes).reset_index()
            formatos.columns = ["Variable","Formato"]
            cuantitativas_bm = list(formatos.loc[formatos["Formato"]!="object",]["Variable"])
            categoricas_bm = list(formatos.loc[formatos["Formato"]=="object",]["Variable"])
            cuantitativas_bm = [x for x in cuantitativas_bm if x not in ["Target"]]
            categoricas_bm = [x for x in categoricas_bm if x not in ["Target"]]

            # Configuración del experimento
            exp_clf101 = setup(data=self.ba_entr,
            target='Target',
            session_id=123,
            train_size=0.7,
            numeric_features = cuantitativas_bm,
            categorical_features = categoricas_bm,
            fix_imbalance=False)
        
        # Comparación de modelos
        best_model = compare_models(sort='AUC') ##accuracy
        return best_model, exp_clf101, cuantitativas, categoricas, c2, cxy, razxy, catxy, cuactxy
        
    def select_model(self, exp_clf101):
        if self.mod == 1:
            dt = create_model('lightgbm')
            
            import pickle
            with open(path + 'best_model.pkl', 'wb') as model_file:
                pickle.dump(dt, model_file)
                
            #### CONTINUAR OPTIMIZACIÓN ####
            # Define the parameter grid for Grid Search
            param_grid_bayesian = {
                'n_estimators': [50,100,200],
                'max_depth': [3,5,7],
                'min_child_samples': [50,150,200]
            }
            # Perform Bayesian Search
            tuned_dt = tune_model(dt, custom_grid=param_grid_bayesian, search_library='scikit-optimize', search_algorithm='bayesian',fold=5)
        elif self.mod == 2:
            dt2 = create_model('rf')  # random forest classifier
            
            import pickle
            with open(path + 'best_model_2.pkl', 'wb') as model_file:
                pickle.dump(dt2, model_file)
                    
            #### CONTINUAR OPTIMIZACION ####
            # Define the parameter grid for Grid Search
            param_grid_bayesian_rf = {
                'n_estimators': [50, 100, 200],
                'max_depth': [3, 5, 7],
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [1, 2, 4],
            }
            # Perform Bayesian Search
            tuned_dt = tune_model(dt2, custom_grid=param_grid_bayesian_rf, search_library='scikit-optimize', search_algorithm='bayesian',fold=5)
        else:
            dt3 = create_model('et')  # extra trees classifier
            
            import pickle
            with open(path + 'best_model_3.pkl', 'wb') as model_file:
                pickle.dump(dt3, model_file)
                    
            #### CONTINUAR OPTIMIZACIÓN ####
            # Define the parameter grid for Grid Search
            param_grid_bayesian_et = {
                'n_estimators': [50, 100, 200],
                'max_depth': [3, 5, 7],
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [1, 2, 4]
            }
            # Perform Bayesian Search
            tuned_dt = tune_model(dt3, custom_grid=param_grid_bayesian_et, search_library='scikit-optimize', search_algorithm='bayesian',fold=5)
        # Visualización exhaustiva del modelo
        evaluate_model(tuned_dt)
                
        # Evaluar el modelo en el conjunto de prueba
        predictions_test = predict_model(tuned_dt)
    
        predictions_train = predict_model(tuned_dt, data=exp_clf101.get_config('X_train'))
    
        y_train = get_config('y_train')
        y_test = get_config('y_test')
    
        from sklearn.metrics import accuracy_score, roc_auc_score
        # Error de entrenamiento
        u1 = accuracy_score(y_train,predictions_train["prediction_label"])
        # Error de test
        u2 = accuracy_score(y_test,predictions_test["prediction_label"])
    
        # Finalizar el modelo
        final_dt = finalize_model(tuned_dt)
        return final_dt, u1, u2
        
    def predict(self, final_dt, cuantitativas, categoricas, c2, cxy, razxy, catxy, cuactxy):
        if self.met == True:
            #### PREDICCIÓN NUEVOS DATOS ####
            # Variables cuantitativas (Activar D1)
            D1 = self.ba_pru.get(cuantitativas).copy()

            # Variables categóricas
            def nombre_(x):
              return "C"+str(x)
            
            D2 = self.ba_pru.get(categoricas).copy()
            for k in categoricas:
              D2[k] = D2[k].map(nombre_)
            D4 = D2.copy()

            
            # Variables al cuadrado (Activar D1)
            cuadrado = [re.findall(r'(.+)_\d+', item) for item in c2]
            cuadrado = [x[0] for x in cuadrado]
            for k in cuadrado:
              D1[k+"_2"] = D1[k] ** 2

            # Interacciones cuantitativas (Activar D1)
            result = [re.findall(r'([A-Za-z\s\(\)0-9]+)', item) for item in cxy]
            for k in result:
              D1[k[0]+"__"+k[1]] = D1[k[0]] * D1[k[1]]

            # Razones
            result2 = [re.findall(r'(.+)__coc__(.+)', item) for item in razxy]
            for k in result2:
              k2 = k[0]
              D1[k2[0]+"__coc__"+k2[1]] = D1[k2[0]] / (D1[k2[1]]+0.01)

            # Interacciones categóricas
            result3 = [re.search(r'([^_]+__[^_]+)', item).group(1).split('__') for item in catxy]
            for k in result3:
              D4[k[0]+"__"+k[1]] = D4[k[0]] + "_" + D4[k[1]]

            # Interacción cuantitativa vs categórica
            D5 = self.ba_pru.copy()
            result4 = [re.search(r'(.+?)_(.+?)_1', item).groups() for item in cuactxy]
            contador = 0
            for k in result4:
              col1, col2 = k[1], k[0] # categórica, cuantitativa
              if contador == 0:
                D51 = pd.get_dummies(D5[col1],drop_first=True)
                for j in D51.columns:
                  D51[j] = D51[j] * D5[col2]
                D51.columns = [col2+"_"+col1+"_"+ str(x) for x in D51.columns]
              else:
                D52 = pd.get_dummies(D5[col1],drop_first=True)
                for j in D52.columns:
                  D52[j] = D52[j] * D5[col2]
                D52.columns = [col2+"_"+col1+"_"+ str(x) for x in D52.columns]
                D51 = pd.concat([D51,D52],axis=1)
              contador = contador + 1

            B1 = pd.concat([D1,D4],axis=1)
            base_modelo2 = pd.concat([B1,D51],axis=1)

            df_test = base_modelo2.copy()
        else:
            df_test = self.ba_pru.copy()
        # Realizar predicciones
        predictions = predict_model(final_dt, data=df_test)

        predictions["sc"] = predictions.apply(lambda row: 1 if row["prediction_score"]<0.9 else 0, axis = 1)

        inverse_category_mapping = {0: 'Graduate', 1: 'Dropout', 2: 'Enrolled'}
        predictions['prediction_label'] = predictions['prediction_label'].map(inverse_category_mapping)
        predictions['prediction_label'] = predictions['prediction_label'].astype('category')
        
        #### ARCHIVO KAGGLE ####
        # Create a DataFrame with 'id' and 'Exited' probabilities
        result = pd.DataFrame({
            'id': self.ba_pru["id"],
            'Target': predictions['prediction_label']
        })

        # Save the result to a CSV file
        result.to_csv(path + 'modelopp.csv', index=False,sep=",")
        return result
    #def evaluate_model(self, u1, u2):
        #return 100 * u2
    def ML_FLOW(self):
        try:
            # Paso 1: Cargar datos
            self.ba_entr, self.ba_pru = self.load_data()
        
            # Paso 2: Preprocesamiento
            self.ba_entr, self.ba_pru = self.preprocessing()
        
            # Paso 3: Entrenamiento del modelo
            best_model, exp_clf101, cuantitativas, categoricas, c2, cxy, razxy, catxy, cuactxy = self.train_model()
         
            # Paso 4: Selección del modelo
            final_dt, u1, u2 = self.select_model(exp_clf101)
        
            # Paso 5: Predicción
            result = self.predict(final_dt, cuantitativas, categoricas, c2, cxy, razxy, catxy, cuactxy)
        
            # Paso 6: Evaluación
            metric = self.evaluate_model(u1, u2)
            return {'success':True,'accuracy':metric}
        except Exception as e:
            return {'success':False,'message':str(e)}

In [48]:
modelo = ML_FLOW_PARCIAL(met=True, mod=1)
salida = modelo.ML_FLOW()

,Description,Value
0,Session id,123
1,Target,Target
2,Target type,Multiclass
3,Original data shape,"(76518, 52)"
4,Transformed data shape,"(76518, 145)"
5,Transformed train set shape,"(53562, 145)"
6,Transformed test set shape,"(22956, 145)"
7,Numeric features,30
8,Categorical features,21
9,Preprocess,True


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8275,0.9429,0.8275,0.8267,0.8263,0.7227,0.7237
1,0.8359,0.9463,0.8359,0.8361,0.8351,0.7365,0.7375
2,0.8297,0.9451,0.8297,0.8284,0.8279,0.7257,0.7270
3,0.8265,0.9425,0.8265,0.8284,0.8260,0.7214,0.7229
4,0.8370,0.9449,0.8370,0.8364,0.8353,0.7374,0.7390
5,0.8295,0.9398,0.8295,0.8276,0.8269,0.7246,0.7265
6,0.8325,0.9449,0.8325,0.8312,0.8306,0.7302,0.7315
7,0.8228,0.9415,0.8228,0.8232,0.8223,0.7160,0.7168
8,0.8290,0.9447,0.8290,0.8305,0.8283,0.7254,0.7268


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8288,0.9437,0.8288,0.8276,0.8274,0.7248,0.7257
1,0.8298,0.9431,0.8298,0.8284,0.8280,0.7259,0.7272
2,0.8333,0.9422,0.8333,0.8314,0.8309,0.7308,0.7325
3,0.8278,0.9429,0.8278,0.8256,0.8258,0.7228,0.7238
4,0.8333,0.9452,0.8333,0.8333,0.8321,0.7319,0.7332
Mean,0.8306,0.9434,0.8306,0.8293,0.8288,0.7272,0.7285
Std,0.0023,0.0010,0.0023,0.0027,0.0023,0.0035,0.0037


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8326,0.9441,0.8326,0.8317,0.8309,0.7303,0.7318


TypeError: 'NoneType' object is not iterable

In [41]:
salida

{'success': False, 'message': "name 'cuantitativas' is not defined"}